In [ ]:
from docx import Document

document = Document('C:/Users/shyamagu/Downloads/test3.docx')
alltext = ""
for i, p in enumerate(document.paragraphs):
    alltext += p.text
    alltext += "\n"

#print(alltext)

In [ ]:
# CSVファイルの読み込み
import pandas as pd
#df = pd.read_csv('../private/testdata.csv', encoding='utf-8')

df = pd.read_csv('C:/Users/xxxxx/Downloads/openaisr2.CSV', encoding='utf-8')
#1件目の本文を取得
text = df.iloc[0]['本文']

import tiktoken

encoding_35 = tiktoken.encoding_for_model("gpt-3.5-turbo-0301")
encoding = tiktoken.get_encoding(encoding_35.name)
num_tokens = len(encoding.encode(text))

print(num_tokens)


In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('AOAI_API_KEY')
openai.api_type = "azure"
openai.api_base = os.getenv('AOAI_ENDPOINT')
openai.api_version = "2023-05-15"

def get_completion_from_messages(messages,temperature=0):
    response = openai.ChatCompletion.create(
        engine=os.getenv('AOAI_MODEL'),
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )

    print(response.usage)

    return response.choices[0].message["content"]

In [ ]:
def formatText(text):
    text = text.replace("\r\n", "\n")
    text = text.replace("\n\n", "\n")
    text = text.replace("  ", " ")
    return text

In [ ]:
text = formatText(text)

encoded = encoding.encode(text)

num_tokens = len(encoded)

print(num_tokens)

In [ ]:
def analyzedText(text):
    system_prompt = """あなたは日本語テキスト文章解析器です。検索インデックス作成のため、入力された文章を検索対象として意味のあるチャンク単位に分割します。
チャンクは極力100文字以上300文字以下になるようにしてください。
なお、入力される文章は長文の一部の可能性があります。

分割した結果は、以下の出力フォーマットに従ってJSONのみ回答してください。
**説明や補足などは不要です。JSONのみ回答してください。**

JSONフォーマット:
[
    {
        "chunk": "チャンク1",
    },
    {
        "chunk": "チャンク2",
    }
]
"""

    user_prompt = text
    
    messages =  [
    {'role':'system', 'content':system_prompt},
    {'role':'user', 'content':user_prompt},
    ]
    response = get_completion_from_messages(messages, temperature=0)
    print(response)
    return response
    #平文のresponseをJSONに変換
    import json
    response_json = json.loads(response)
    return response_json


In [ ]:
result_str = []
for i in range(0, num_tokens, 1800):
    start = 0
    if i-400 >= 0:
        start = i-400
    text = encoding.decode(encoded[start:i+1800])
    res = analyzedText(text)
    result_str.append(res)